In [1]:
!pip install -U transformers datasets

In [2]:
!pip install -U transformers[torch] accelerate


In [3]:
!pip install -U transformers[torch] accelerate

In [4]:
!pip install -U accelerate


In [5]:
!pip install -U accelerate


In [6]:
!pip install tf-keras


  Using cached keras-3.8.0-py3-none-any.whl.metadata (5.8 kB)
Using cached keras-3.8.0-py3-none-any.whl (1.3 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.14.0
    Uninstalling keras-2.14.0:
      Successfully uninstalled keras-2.14.0


In [7]:
!pip install keras==2.14.0


  Using cached keras-2.14.0-py3-none-any.whl.metadata (2.4 kB)
Using cached keras-2.14.0-py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: keras 3.8.0
    Uninstalling keras-3.8.0:
      Successfully uninstalled keras-3.8.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.18.0 requires keras>=3.5.0, but you have keras 2.14.0 which is incompatible.


In [8]:
!pip install -U transformers


In [9]:
!pip install -U torch


In [10]:
!pip uninstall transformers accelerate torch -y
!pip install transformers[torch] accelerate torch

Found existing installation: transformers 4.48.3
Uninstalling transformers-4.48.3:
  Successfully uninstalled transformers-4.48.3
Found existing installation: accelerate 1.3.0
Uninstalling accelerate-1.3.0:
  Successfully uninstalled accelerate-1.3.0
Found existing installation: torch 2.6.0
Uninstalling torch-2.6.0:
  Successfully uninstalled torch-2.6.0
  Using cached accelerate-1.3.0-py3-none-any.whl.metadata (19 kB)
  Using cached torch-2.6.0-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached transformers-4.48.3-py3-none-any.whl.metadata (44 kB)
Using cached accelerate-1.3.0-py3-none-any.whl (336 kB)
Using cached torch-2.6.0-cp312-cp312-win_amd64.whl (204.1 MB)
Using cached transformers-4.48.3-py3-none-any.whl (9.7 MB)


In [11]:
import os
import requests
from PIL import Image
from io import BytesIO
import torch
from transformers import AutoImageProcessor, AutoModelForImageClassification, Trainer, TrainingArguments, pipeline
from datasets import Dataset, concatenate_datasets
import pandas as pd
from torchvision import transforms
from tqdm import tqdm
import gc
import pyarrow as pa
import pickle


In [12]:
import accelerate
print(accelerate.__version__)

1.3.0


In [13]:
print(torch.__version__)

2.6.0+cpu


In [14]:
WorkDir = "C:\\Users\\SMHRD\\Desktop\\5g\\5G\\back\projectmodel\\project_model\\image_classification"

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\SMHRD\AppData\Local\Temp\ipykernel_13244\3255522326.py:1: SyntaxWarning: invalid escape sequence '\p'
  WorkDir = "C:\\Users\\SMHRD\\Desktop\\5g\\5G\\back\projectmodel\\project_model\\image_classification"


In [15]:
crawled_dir = os.path.join(WorkDir, "crawled_images")

In [16]:
# CSV 파일 로드
df = pd.read_csv(os.path.join(WorkDir, 'aaa.csv'))

In [17]:
#이미지 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [18]:
# ✅ 중복 방지 및 스타일 데이터셋 생성 (배치 변환 적용)
data_style = []
seen_images = set()
batch_size = 1000  # 배치 크기 조정 가능
style_datasets = []

for row in tqdm(df.itertuples(), total=len(df)):
    img_path = os.path.join(crawled_dir, f'image_{row.ID}.jpg')

    if row.ID in seen_images:
        continue

    seen_images.add(row.ID)

    try:
        img = Image.open(img_path).convert("RGB")
        img_tensor = transform(img).numpy()  # ✅ PyArrow 문제 방지를 위해 `numpy()`로 변환
        label = row.STYLE
        data_style.append({"image": img_tensor, "label": label})
    
        # ✅ 배치 단위로 변환하여 메모리 절약
        if len(data_style) >= batch_size:
            dataset_batch = Dataset.from_list(data_style)
            style_datasets.append(dataset_batch)
            data_style = []  # 리스트 초기화
    except Exception as e:
        print(f"🔴 이미지 로드 실패: {img_path} ({e})")

100%|██████████| 3783/3783 [02:40<00:00, 23.63it/s]


In [19]:
# ✅ 마지막 남은 데이터 변환
if data_style:
    dataset_batch = Dataset.from_list(data_style)
    style_datasets.append(dataset_batch)


In [20]:
# ✅ 전체 데이터셋 합치기
style_dataset = concatenate_datasets(style_datasets)
style_dataset = style_dataset.train_test_split(test_size=0.2)

In [21]:
# ✅ Hugging Face 방식으로 저장
dataset_path = os.path.join(WorkDir, 'style_dataset')
style_dataset.save_to_disk(dataset_path)
print(f"✅ 스타일 데이터셋 저장 완료: {dataset_path}")

Saving the dataset (0/4 shards):   0%|          | 0/3026 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/757 [00:00<?, ? examples/s]

✅ 스타일 데이터셋 저장 완료: C:\Users\SMHRD\Desktop\5g\5G\back\projectmodel\project_model\image_classification\style_dataset


In [22]:
# ✅ 모델 설정
model_name = "google/vit-base-patch16-224-in21k"
processor = AutoImageProcessor.from_pretrained(model_name)

id2label = {id: label for id, label in enumerate(df["STYLE"].unique())}
label2id = {label: id for id, label in id2label.items()}

model_style = AutoModelForImageClassification.from_pretrained(
    model_name, num_labels=len(id2label), id2label=id2label, label2id=label2id
)

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# 학습 설정
training_args_style = TrainingArguments(
    output_dir=os.path.join(WorkDir, 'style_result'),
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    save_strategy="epoch",
    remove_unused_columns=False,
    logging_dir=None,
    report_to=[],
)

c:\Users\SMHRD\anaconda3\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [24]:
# ✅ 데이터 로더 (Tensor 변환)
def collate_fn_style(batch):
    images = torch.stack([torch.tensor(b["image"]) for b in batch]).to(torch.uint8)
    labels = torch.tensor([model_style.config.label2id[b["label"]] for b in batch])
    return {"pixel_values": images, "labels": labels}


In [25]:
# ✅ Trainer 설정 및 학습 실행
trainer_style = Trainer(
    model=model_style,
    args=training_args_style,
    train_dataset=style_dataset['train'],
    eval_dataset=style_dataset['test'],
    data_collator=collate_fn_style
)

In [26]:
# 학습 실행
trainer_style.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# ✅ 스타일 분류 파이프라인 생성
style_classifier = pipeline(
    "image-classification",
    model=model_style,
    feature_extractor=processor
)

In [ ]:
# ✅ 스타일 예측 함수
def predict_image_style(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    predictions = style_classifier(**inputs)
    return predictions[0]['label']

In [ ]:
# ✅ 일정 데이터셋 변환 (배치 적용)
data_schedule = []
seen_images = set()
schedule_datasets = []

for row in tqdm(df.itertuples(), total=len(df)):
    img_path = os.path.join(crawled_dir, f'image_{row.ID}.jpg')

    if row.ID in seen_images:
        continue

    seen_images.add(row.ID)

    try:
        img = Image.open(img_path).convert("RGB")
        img_tensor = transform(img).numpy()  # ✅ `numpy()` 변환
        label = row.SCHEDULE
        data_schedule.append({"image": img_tensor, "label": label})

        if len(data_schedule) >= batch_size:
            dataset_batch = Dataset.from_list(data_schedule)
            schedule_datasets.append(dataset_batch)
            data_schedule = []
    except Exception as e:
        print(f"🔴 이미지 로드 실패: {img_path} ({e})")


In [ ]:
if data_schedule:
    dataset_batch = Dataset.from_list(data_schedule)
    schedule_datasets.append(dataset_batch)

In [ ]:
schedule_dataset = concatenate_datasets(schedule_datasets)
schedule_dataset = schedule_dataset.train_test_split(test_size=0.2)

In [ ]:
# ✅ Hugging Face 방식으로 저장
dataset_path = os.path.join(WorkDir, 'schedule_dataset')
schedule_dataset.save_to_disk(dataset_path)
print(f"✅ 일정 데이터셋 저장 완료: {dataset_path}")


In [ ]:
# ✅ 일정 모델 로드
model_schedule = AutoModelForImageClassification.from_pretrained(
    model_name, num_labels=len(df["SCHEDULE"].unique())
)

In [ ]:
# ✅ 학습 설정
training_args_schedule = TrainingArguments(
    output_dir=os.path.join(WorkDir, 'schedule_result'),
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    save_strategy="epoch",
    remove_unused_columns=False,
    logging_dir=None,
    report_to=[],
)

In [ ]:
# ✅ 데이터 로더 (Tensor 변환)
def collate_fn_schedule(batch):
    images = torch.stack([torch.tensor(b["image"]) for b in batch]).to(torch.uint8)
    labels = torch.tensor([df["SCHEDULE"].unique().tolist().index(b["label"]) for b in batch])
    return {"pixel_values": images, "labels": labels}


In [ ]:
trainer_schedule = Trainer(
    model=model_schedule,
    args=training_args_schedule,
    train_dataset=schedule_dataset['train'],
    eval_dataset=schedule_dataset['test'],
    data_collator=collate_fn_schedule
)

In [ ]:
trainer_schedule.train()

In [ ]:
# ✅ 일정 분류 파이프라인 생성
schedule_classifier = pipeline(
    "image-classification",
    model=model_schedule,
    feature_extractor=processor
)


In [ ]:
# ✅ 일정 예측 함수
def predict_image_schedule(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    predictions = schedule_classifier(**inputs)
    return predictions[0]['label']